In [1]:
import pandas as pd
import tweepy
import os

#setup tweepy Oauth
auth = tweepy.OAuthHandler(
    'YMaRCwObPMfcEvcqDSMaEza4x', 
    'qmawQuA8p6Rxvp9rrLREUkaLVRLRWpUdOE9GaI2BLpxntq8iat'
)
auth.set_access_token(
    '961387368463781888-6NYJVeWtVhbXVU12qvRno4kuq43492N',
    '5DsU7UfhghntczyLNsijssrfZVpcsOe8Rbo24Ighjnzwo'
)
api=tweepy.API(auth)

In [2]:
def csvs_to_txt(inputs, output, column, cst=False):
    out_file = open(output, "w")
    for f in files:
        if not cst:
            df_tweets=pd.read_csv(f, error_bad_lines=False)
        else:
            df_tweets=pd.read_table(f, sep='\t')
        for text in df_tweets[column]:
            out_file.write(str(text).replace('\r', '').replace('\n',' ')+"\n")
    out_file.close

In [3]:
#CrisisLexT6
out = "tweets_CrisisLexT6.txt"
files = []
files.append("tweets/CrisisLexT6/2012_Sandy_Hurricane/2012_Sandy_Hurricane-ontopic_offtopic.csv")
files.append("tweets/CrisisLexT6/2013_Alberta_Floods/2013_Alberta_Floods-ontopic_offtopic.csv")
files.append("tweets/CrisisLexT6/2013_Boston_Bombings/2013_Boston_Bombings-ontopic_offtopic.csv")
files.append("tweets/CrisisLexT6/2013_Oklahoma_Tornado/2013_Oklahoma_Tornado-ontopic_offtopic.csv")
files.append("tweets/CrisisLexT6/2013_Queensland_Floods/2013_Queensland_Floods-ontopic_offtopic.csv")
files.append("tweets/CrisisLexT6/2013_West_Texas_Explosion/2013_West_Texas_Explosion-ontopic_offtopic.csv")
csvs_to_txt(files, out, " tweet")

In [3]:
import time
#Extract from ids
def ids_to_txt(id_txt, output):
    with open(id_txt, 'r') as in_file:
        with open(output, 'w') as out_file:
            count=0
            count_t=0
            t=2500
            ids=[]
            for line in in_file:
                ids.append(int(line))
                if len(ids)>=100:
                    count_t+=100
                    try:
                        status=api.statuses_lookup(ids, False, True, False)
                    except tweepy.TweepError:
                        print "Waiting for API Cooldown"
                        while(True):
                            time.sleep(5 * 60)
                            try:
                                status=api.statuses_lookup(ids, False, True, False)
                                break
                            except tweepy.TweepError:
                                continue
                    count+=len(status)
                    for s in status:
                        out_file.write(s.text.encode('ascii', 'ignore').replace('\r', '').replace('\n',' ')+"\n")
                    ids=[]
                    if count_t>=t:
                        print str(count)+'/'+str(count_t)
                        t+=2500

In [6]:
#Train Model
from gensim.models import Word2Vec, FastText, KeyedVectors
from gensim.models.keyedvectors import Word2VecKeyedVectors, FastTextKeyedVectors

from glove import Glove, Corpus
import re

class TextIterator(object):
    def __init__(self, dirname):
        self.dirname=dirname
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                a=line.lower()
                a=re.sub(r'https?:\/\/[^\s]*','',a)
                a=re.findall(r'\w+(?:[-\']\w+)?',a)
                yield a

In [6]:
#CrisisLexT26

out="tweets_CrisisLexT26.txt"
dirs = os.listdir('tweets/CrisisLexT26')
dirs.remove('README.md')
files = ['tweets/CrisisLexT26/'+n+"/"+n+"-tweets_labeled.csv" for n in dirs]
csvs_to_txt(files, out, " Tweet Text")

In [19]:
#kaggle - Hurricane Harvey

out="tweets_kaggle_harvey.txt"
files = ['tweets/hurricaneharvey/'+n for n in os.listdir('tweets/hurricaneharvey/')]

# a=pd.read_table(files[1], sep=',')
# a['Tweet'].head()[2]

csvs_to_txt(files, out, "Tweet")

In [30]:
#ISCRAM18_dataset - Harvey missing IDs
out="tweets_ISCRAM18_harvey_missing_ids.txt"
files=['tweets/ISCRAM18_datasets/Harvey_missing_ids_50K.csv']
csvs_to_txt(files, out, "Tweet", True)

In [43]:
#ISCRAM2013_dataset
out="tweets_ISCRAM2013.txt"
dirs=os.listdir('tweets/ISCRAM2013_dataset')
dirs.remove('Terms of use.txt')
dirs.remove('README.txt')
files=['tweets/ISCRAM2013_dataset/'+n+"/"+os.listdir('tweets/ISCRAM2013_dataset/'+n)[0] for n in dirs]
csvs_to_txt(files, out, "tweet")

In [44]:
#figure-eight
out="tweets_figure_eight.txt"
files=["tweets/socialmedia-disaster-tweets-DFE.csv"]
csvs_to_txt(files, out, "text")

In [ ]:
#ISCRAM18 datasets
out=[
    "tweets_ISCRAM18_Harvey.txt",
    "tweets_ISCRAM18_Irma.txt",
    "tweets_ISCRAM18_Maria.txt"
]
files=[
    'tweets/ISCRAM18_datasets/Harvey_tweet_ids.txt',
    'tweets/ISCRAM18_datasets/Irma_tweet_ids.txt',
    'tweets/ISCRAM18_datasets/Maria_tweet_ids.txt'
]
for i, f in enumerate(files):
    ids_to_txt(files[i], out[i])

In [12]:
tdir="tweets/CrisisLexT26/"
ids="tweets/CL26_tweet_ids.txt"
out="tweets_CrisisLexT26_ids.txt"
dirs = os.listdir('tweets/CrisisLexT26')
dirs.remove('README.md')
files = ['tweets/CrisisLexT26/'+n+"/"+n+"-tweetids_entire_period.csv" for n in dirs]
csvs_to_txt(files, ids, " Tweet-ID")
ids_to_txt(ids,out)

1927/2500
3295/5000
5053/7500
6859/10000
8085/12500
9648/15000
11338/17500
12924/20000
14402/22500
16149/25000
17761/27500
19275/30000
Waiting for API Cooldown
20682/32500
22060/35000
23423/37500
24715/40000
26097/42500
27538/45000
28988/47500
30377/50000
31779/52500
33157/55000
34576/57500
35970/60000
37357/62500
38761/65000
40188/67500
41559/70000
42935/72500
44303/75000
45632/77500
46967/80000
48221/82500
49555/85000
50861/87500
52072/90000
53399/92500
54748/95000
55958/97500
57218/100000
58508/102500
59842/105000
61154/107500
62511/110000
63880/112500
65305/115000
66747/117500
68239/120000
Waiting for API Cooldown
69637/122500
71177/125000
72787/127500
74345/130000
75818/132500
77479/135000
78990/137500
80671/140000
82320/142500
83904/145000
85513/147500
86954/150000
88527/152500
90029/155000
91617/157500
93308/160000
95030/162500
96820/165000
98566/167500
100114/170000
101817/172500
103545/175000
105403/177500
107162/180000
108855/182500
110718/185000
112578/187500
114402/190000
1

In [34]:
directory="data/tweet_txts/"
sents=TextIterator(directory)

model=Word2Vec(sents, size=300, workers=4, min_count=1)
word_vectors=model.wv
word_vectors.save("word2vec_tweets_300_2.wv")
# model=Word2Vec(sents, size=100, workers=4)
# word_vectors=model.wv
# word_vectors.save("word2vec_tweets_100.wv")
# model=Word2Vec(sents, size=500, workers=4)
# word_vectors=model.wv
# word_vectors.save("word2vec_tweets_500.wv")
# model=FastText(sents, size=300, workers=4)
# word_vectors=model.wv
# word_vectors.save("fasttext_300.wv")
# glove= Glove(no_components=300)
# corpus=Corpus()
# corpus.fit(sents, window=5)
# #glove.fit(sents, epochs=5)

# glove= Glove(no_components=300)
# glove.fit(corpus.matrix, epochs=5, verbose=True)
# glove.add_dictionary(corpus.dictionary)

# glove.save('glove.wv')

In [35]:
g_vec=KeyedVectors.load_word2vec_format(
            'GoogleNews-vectors-negative300.bin', 
            binary=True, limit=200000)
t_vec=Word2VecKeyedVectors.load("word2vec_tweets_300.wv")

In [118]:
w1, w2 = "hurricane", "storm"
print g_vec.similarity(w1,w2)
print t_vec.similarity(w1,w2)

0.7321895
0.40604332
